In [1]:
import os
import pandas as pd
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from sklearn.metrics import fbeta_score
from tqdm import tqdm
from sklearn.utils import shuffle
import cv2
from PIL import Image
from tensorflow import keras
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential 
import seaborn as sns
import tensorflow as tf
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from keras.optimizers import Adam, SGD
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau, History

In [3]:
tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
tf.config.experimental_connect_to_cluster(tpu)
tf.tpu.experimental.initialize_tpu_system(tpu)

ValueError: Please provide a TPU Name to connect to.

In [ ]:
tpu_strategy = tf.distribute.experimental.TPUStrategy(tpu)


In [4]:
train_label = pd.read_csv('../input/planets-dataset/planet/planet/train_classes.csv')
sam_sub = pd.read_csv('../input/planets-dataset/planet/planet/sample_submission.csv')
train_label.head()

,image_name,tags
0,train_0,haze primary
1,train_1,agriculture clear primary water
2,train_2,clear primary
3,train_3,clear primary
4,train_4,agriculture clear habitation primary road


In [5]:
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
#onehot=OneHotEncoder()
encoder = LabelEncoder()
#encoder.fit(train_label['tags'])
label_maps = pd.DataFrame()
label_maps['tags'] = ['agriculture', 'artisinal_mine', 'bare_ground','blooming','blow_down','clear','cloudy','conventional_mine','cultivation','habitation','haze', 'partly_cloudy','primary','road','selective_logging','slash_burn','water']
label_maps['map'] = encoder.fit_transform(label_maps['tags'])
label_maps

,tags,map
0,agriculture,0
1,artisinal_mine,1
2,bare_ground,2
3,blooming,3
4,blow_down,4
5,clear,5
6,cloudy,6
7,conventional_mine,7
8,cultivation,8
9,habitation,9


In [6]:
label_map = {'agriculture': 0,
 'artisinal_mine': 1,
 'bare_ground': 2,
 'blooming': 3,
 'blow_down': 4,
 'clear': 5,
 'cloudy': 6,
 'conventional_mine': 7,
 'cultivation': 8,
 'habitation': 9,
 'haze': 10,
 'partly_cloudy': 11,
 'primary': 12,
 'road': 13,
 'selective_logging': 14,
 'slash_burn': 15,
 'water': 16}


In [8]:
X = []
Y = []
train_label = shuffle(train_label,random_state=0)
for image_name, tags in tqdm(train_label.values, miniters=400):
    arr = cv2.imread('../input/planets-dataset/planet/planet/train-jpg/{}.jpg'.format(image_name), cv2.IMREAD_UNCHANGED)
    targets = np.zeros(17)
    for t in tags.split(' '):
      targets[label_map[t]] = 1 
    arr = cv2.resize(arr, (64, 64))
    X.append(arr)
    Y.append(targets)
            

X = np.array(X, np.float16)/255.0

100%|██████████| 40479/40479 [01:21<00:00, 497.71it/s]


In [9]:
from sklearn.model_selection import train_test_split

X = np.array(X)
Y = np.array(Y)
x_train, x_val, y_train, y_val = train_test_split(X, Y, test_size = 0.2, shuffle = True, random_state = 1)

print(x_train.shape, y_train.shape, x_val.shape, y_val.shape)

(32383, 64, 64, 3) (32383, 17) (8096, 64, 64, 3) (8096, 17)


In [10]:
import gc
del X, Y
gc.collect()

52

In [11]:
test = sam_sub[0 : 40669]
files = sam_sub[40669 : ]

In [12]:
import gc
gc.collect()

40

In [13]:
import numpy as np
from keras import backend as K
from sklearn.metrics import fbeta_score

def fbeta(y_true, y_pred, threshold_shift=0):
    beta = 2

    # just in case of hipster activation at the final layer
    y_pred = K.clip(y_pred, 0, 1)

    # shifting the prediction threshold from .5 if needed
    y_pred_bin = K.round(y_pred + threshold_shift)

    tp = K.sum(K.round(y_true * y_pred_bin)) + K.epsilon()
    fp = K.sum(K.round(K.clip(y_pred_bin - y_true, 0, 1)))
    fn = K.sum(K.round(K.clip(y_true - y_pred, 0, 1)))

    precision = tp / (tp + fp)
    recall = tp / (tp + fn)

    beta_squared = beta ** 2
    return (beta_squared + 1) * (precision * recall) / (beta_squared * precision + recall + K.epsilon())

In [24]:
# base model. Feel free to try out other architectures and ideas to improve fbeta score


from keras.layers import Conv2D, Dense, LSTM, Flatten, MaxPooling2D, BatchNormalization, Dropout
from keras.optimizers import Adam, RMSprop, SGD

#with tpu_strategy.scope():
model = keras.Sequential()
model.add(Conv2D(64, 5, 2, activation = "relu", input_shape = (64, 64, 3)))
model.add(MaxPooling2D())
model.add(Conv2D(128, 5, 2, activation = "relu"))
model.add(MaxPooling2D())
model.add(Flatten())
model.add(Dense(512, activation = "relu"))
model.add(Dense(17, activation = "sigmoid"))
model.compile(loss = "binary_crossentropy", optimizer = Adam(), metrics = [fbeta])
model.fit(x_train, y_train, validation_data = (x_val, y_val), epochs = 45, batch_size = 64)

Epoch 1/45
506/506 [==============================] - 4s 7ms/step - loss: 0.2251 - fbeta: 0.6368 - val_loss: 0.2098 - val_fbeta: 0.6411
Epoch 2/45
506/506 [==============================] - 3s 6ms/step - loss: 0.1945 - fbeta: 0.6891 - val_loss: 0.1852 - val_fbeta: 0.6913
Epoch 3/45
506/506 [==============================] - 3s 6ms/step - loss: 0.1767 - fbeta: 0.7237 - val_loss: 0.1661 - val_fbeta: 0.7449
Epoch 4/45
506/506 [==============================] - 3s 6ms/step - loss: 0.1624 - fbeta: 0.7503 - val_loss: 0.1617 - val_fbeta: 0.7753
Epoch 5/45
506/506 [==============================] - 3s 7ms/step - loss: 0.1553 - fbeta: 0.7628 - val_loss: 0.1519 - val_fbeta: 0.7592
Epoch 6/45
506/506 [==============================] - 3s 6ms/step - loss: 0.1493 - fbeta: 0.7719 - val_loss: 0.1494 - val_fbeta: 0.7583
Epoch 7/45
506/506 [==============================] - 3s 7ms/step - loss: 0.1464 - fbeta: 0.7761 - val_loss: 0.1450 - val_fbeta: 0.7765
Epoch 8/45
506/506 [============================

In [26]:
import gc
gc.collect()

20

In [27]:
#with tpu_strategy.scope():
test_loss, test_accuracy = model.evaluate(x_val, y_val)
print('Test loss: {}'.format(test_loss))
print('Test accuracy: {}'.format(test_accuracy))

253/253 [==============================] - 1s 3ms/step - loss: 0.1986 - fbeta: 0.8082
Test loss: 0.19860467314720154
Test accuracy: 0.8081573247909546


In [29]:
import gc
gc.collect()

20

In [30]:
#with tpu_strategy.scope():
model.evaluate(x_val, y_val)
print("train fscore: ", fbeta_score(y_train, np.round(model.predict(x_train)), 2,average = 'weighted'))
print("val fscore: ", fbeta_score(y_val, np.round(model.predict(x_val)), 2, average = 'weighted'))

253/253 [==============================] - 1s 3ms/step - loss: 0.1986 - fbeta: 0.8082
train fscore:  0.9057846207582922
val fscore:  0.8011339071755773


In [48]:
import gc
del x_train, y_train, x_val, y_val
gc.collect()

20

In [49]:
#with tpu_strategy.scope():   
test_img = []

for image_name, tags in tqdm(test.values, miniters=1000):
    arr = cv2.imread('../input/planets-dataset/planet/planet/test-jpg/{}.jpg'.format(image_name))
    test_img.append(cv2.resize(arr, (64, 64)))

for image_name, tags in tqdm(files.values, miniters=1000):
    arr = cv2.imread('../input/planets-dataset/test-jpg-additional/test-jpg-additional/{}.jpg'.format(image_name))
    test_img.append(cv2.resize(arr, (64, 64)))

test_img = np.array(test_img, np.float16)/255.0

100%|██████████| 20522/20522 [01:04<00:00, 315.94it/s]


In [51]:
import gc
gc.collect()

20

In [52]:
#with tpu_strategy.scope():
yres = []
predictions = model.predict(test_img, batch_size = 64, verbose = 2)
yres.append(predictions)

957/957 - 2s


In [54]:
import gc
gc.collect()

20

In [55]:
sub = np.array(yres[0])
for i in range (1, len(yres)):
    sub += np.array(yres[i])
sub = pd.DataFrame(sub, columns = label_map)


In [56]:
sub

,agriculture,artisinal_mine,bare_ground,blooming,blow_down,clear,cloudy,conventional_mine,cultivation,habitation,haze,partly_cloudy,primary,road,selective_logging,slash_burn,water
0,0.036243,1.623822e-09,5.472098e-04,1.792209e-02,9.458773e-05,9.979954e-01,4.846733e-05,8.412109e-09,0.007351,6.484619e-03,1.348638e-03,8.776189e-05,0.999797,5.199452e-02,4.914914e-03,8.964487e-07,0.019867
1,0.032196,4.245196e-10,5.076811e-05,1.249706e-03,7.604660e-03,5.550858e-01,9.866937e-09,4.498660e-09,0.007909,3.337563e-03,2.669762e-05,5.529774e-01,0.999999,5.135342e-02,9.452160e-02,8.616440e-06,0.020143
2,0.000031,5.333672e-11,1.229562e-10,5.127135e-17,2.482491e-10,1.383804e-07,8.158782e-14,4.549754e-20,0.004295,3.956552e-09,2.072174e-06,9.999998e-01,1.000000,9.108058e-08,1.540028e-15,4.781141e-13,0.035616
3,0.189300,6.485872e-09,2.123604e-04,2.336223e-02,3.841462e-03,4.977142e-01,1.157311e-06,6.099198e-05,0.288850,1.028778e-03,3.250281e-04,4.547449e-01,0.999994,1.487549e-03,2.456338e-03,8.603365e-04,0.006519
4,0.026443,3.934880e-12,2.498247e-03,1.555433e-14,4.432799e-14,7.353558e-01,5.129969e-01,2.930768e-14,0.000128,3.402748e-02,2.814914e-10,5.563788e-02,0.848086,2.199147e-02,6.142937e-13,1.132640e-12,0.003643
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61186,0.003595,1.281794e-14,1.210457e-07,2.324145e-17,5.573845e-17,3.049501e-04,3.010385e-01,2.949783e-12,0.000012,9.238608e-01,9.284815e-05,4.260183e-02,0.936278,9.679334e-01,2.815827e-10,3.082971e-13,0.005654
61187,0.938803,1.348563e-12,2.392159e-05,3.341211e-15,4.751758e-28,9.997984e-01,8.595878e-16,3.232509e-10,0.012042,2.256647e-04,3.503794e-09,2.286025e-07,1.000000,1.851710e-04,3.209691e-17,2.031361e-07,0.903542
61188,0.055332,5.960421e-11,1.429841e-05,2.026015e-03,2.029463e-02,9.925544e-01,1.289769e-08,1.357692e-08,0.014210,1.043600e-03,4.147082e-05,1.053357e-03,1.000000,1.107821e-01,2.341894e-03,7.575504e-07,0.017077
61189,0.008452,8.813649e-12,3.065596e-07,7.616413e-12,2.290952e-16,9.660793e-04,8.812004e-01,1.264253e-15,0.002342,7.119397e-04,8.498582e-02,8.860142e-05,0.109450,8.340684e-03,5.684003e-11,1.593103e-13,0.025880


In [57]:

preds = []
for i in tqdm(range(sub.shape[0]), miniters=1000):
    a = sub.loc[[i]]
    a = a.apply(lambda x: x > 0.2, axis=1)
    a = a.transpose()
    a= a.loc[a[i] == True]
    ' '.join(list(a.index))
    preds.append(' '.join(list(a.index)))

sam_sub['tags'] = preds
sam_sub.to_csv('subm.csv', index=False)

  0%|          | 0/61191 [00:00<?, ?it/s]


KeyError: "None of [Int64Index([0], dtype='int64')] are in the [columns]"